# Step 4 - Analysis of bicycle network results
## Project: Growing Urban Bicycle Networks

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency
* overlap with existing networks

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2021-11-22

## Preliminaries

### Parameters

In [1]:
debug = False # If True, will produce plots and/or verbose output to double-check
rerun_existing = False # If True, will re-run the costly analysis of existing infra even if files already exist.
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [2]:
%run -i path.py
#%run -i setup.py
%run -i setupCPH.py
if not debug: # Only do this if sure the code is bug-free!
    warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -n -v -m -g -iv

Loaded PATH.

Setup finished.

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 8.5.0

Compiler    : Clang 9.0.1 
OS          : Darwin
Release     : 18.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: 622e89ad3e22ced847c13f6f0b356976fa1617ba

sklearn   : 1.1.2
igraph    : 0.9.1
osgeo     : 3.2.1
matplotlib: 3.6.0
osmnx     : 0.16.2
networkx  : 2.8.6
shapely   : 1.8.4
csv       : 1.0
fiona     : 1.8.21
pandas    : 1.4.4
geopandas : 0.11.1
numpy     : 1.23.3
pyproj    : 3.4.0
watermark : 2.3.1
geojson   : 2.5.0
sys       : 3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:56:27) 
[Clang 9.0.1 ]



### Functions

In [3]:
%run -i functions.py

Loaded functions.



# Join attributes on all networks

In [107]:
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_length_attr.pkl", 'rb') as f:
    unassigned_length_attr = pickle.load(f)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_pop_den_attr.pkl", 'rb') as f:
    unassigned_pop_den_attr = pickle.load(f)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_bcount_attr.pkl", 'rb') as q:
    unassigned_bcount_attr = pickle.load(q)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_bcount_pop_den_att.pkl", 'rb') as q:
    unassigned_bcount_pop_den_attr = pickle.load(q)

## Join new attributes to all networks on geometry

In [110]:
with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_carall_edges.zip", 'r') as zip_ref:
    zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

carall_edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_carall_edges.csv")

for network in networktypesdata:
    print(network)
    
    with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.zip", 'r') as zip_ref:
        zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

    biketrack_edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.csv")


    #the initial edges are joined with the new generalised counts
    result = pd.merge(biketrack_edges, carall_edges[['geometry','length_attr','bcount_attr','pop_den_attr','bcount_pop_den_attr']],how = 'left',on='geometry')
    #left_on=['u','v'], right_on = ['u','v']

    for i in range(len(result)):
        length_attr = result['length_attr'].iloc[i]
        bcount_attr = result['bcount_attr'].iloc[i]
        pop_den_attr = result['pop_den_attr'].iloc[i]
        bcount_pop_den_attr = result['bcount_pop_den_attr'].iloc[i]
        #if num is NaN
        if length_attr != length_attr:
            result['length_attr'].iloc[i]= unassigned_length_attr
        if bcount_attr != bcount_attr:
            result['bcount_attr'].iloc[i]= unassigned_bcount_attr
        if pop_den_attr != pop_den_attr:
            result['pop_den_attr'].iloc[i]= unassigned_pop_den_attr
        if bcount_pop_den_attr != bcount_pop_den_attr:
            result['bcount_pop_den_attr'].iloc[i]= unassigned_bcount_pop_den_attr
            
    result.to_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.csv")
    compress_file("../../bikenwgrowth_external/data/copenhagen/","copenhagen_"+network+"_edges")
    


biketrack
bikeable
biketrackcarall


In [ ]:
for network in networktypesdata:
    print(network)
    
    with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.zip", 'r') as zip_ref:
        zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")
    with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_simplified_edges.zip", 'r') as zip_ref:
        zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

    edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.csv")
    edges_simplified = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_simplified_edges.csv")

In [4]:

with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+"carall"+"_edges.zip", 'r') as zip_ref:
    zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")
with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+"carall"+"_simplified_edges.zip", 'r') as zip_ref:
    zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+"carall"+"_edges.csv")
edges_simplified = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+"carall"+"_simplified_edges.csv")











In [5]:
len(edges)

65752

In [161]:

i=0
for i in range(len(edges)):
    edges_id = edges.iloc[i].osmid
    for v in range(len(edges_simplified)):
        edges_simplified_id = edges_simplified.iloc[v].osmid
        if edges_id == edges_simplified_id:
            i+=1
    
     

KeyboardInterrupt: 

In [ ]:
i

In [ ]:
edges.iloc[7]

In [162]:
df = pd.DataFrame()
i=0
for i in range(len(edges)):
    edges_id = edges.iloc[i].osmid
    for v in range(len(edges_simplified)):
        edges_simplified_id = edges_simplified.iloc[v].osmid
        if edges_id == edges_simplified_id:
            df.append(edges.iloc[i])
            i+=1
            print(i)

KeyboardInterrupt: 

In [32]:
s1 = pd.merge(edges, edges_simplified, how='right', on=['u','v'])

In [33]:
s1

,osmid_x,oneway_x,name_x,highway_x,maxspeed_x,length_x,length_attr_x,bcount_attr_x,pop_den_attr_x,bcount_pop_den_attr_x,...,geometry_y,lanes_y,ref_y,bridge_y,tunnel_y,access_y,junction_y,width_y,service_y,key_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"LINESTRING (12.5629057 55.6739848, 12.5630034 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"LINESTRING (12.5629057 55.6739848, 12.5633457 ...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"LINESTRING (12.5584685 55.6619841, 12.5581719 ...",2,O2,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6.834636e+08,True,NaN,primary_link,50.0,8.591,0.112098,0.113953,0.101020,0.114735,...,"LINESTRING (12.5615093 55.6630978, 12.5616155 ...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,6.834636e+08,True,NaN,primary_link,50.0,8.591,0.112098,0.113953,0.101020,0.114735,...,"LINESTRING (12.5615093 55.6630978, 12.5615457 ...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19554,1.131852e+09,True,Strandvænget,primary,60.0,6.446,0.109070,0.113953,0.101544,0.115251,...,"LINESTRING (12.5832522 55.716637, 12.5831707 5...",3,O2,NaN,NaN,NaN,NaN,NaN,NaN,0
19555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"LINESTRING (12.5832522 55.716637, 12.5834256 5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19556,1.131852e+09,True,Strandvænget,tertiary,50.0,8.028,0.111303,0.113953,0.101923,0.115624,...,"LINESTRING (12.5831349 55.7166079, 12.5832522 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"LINESTRING (12.5831349 55.7166079, 12.583198 5...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [18]:
edges_simplified['geometry']


0        LINESTRING (12.5629057 55.6739848, 12.5630034 ...
1        LINESTRING (12.5629057 55.6739848, 12.5633457 ...
2        LINESTRING (12.5584685 55.6619841, 12.5581719 ...
3        LINESTRING (12.5615093 55.6630978, 12.5616155 ...
4        LINESTRING (12.5615093 55.6630978, 12.5615457 ...
                               ...                        
19554    LINESTRING (12.5832522 55.716637, 12.5831707 5...
19555    LINESTRING (12.5832522 55.716637, 12.5834256 5...
19556    LINESTRING (12.5831349 55.7166079, 12.5832522 ...
19557    LINESTRING (12.5831349 55.7166079, 12.583198 5...
19558    LINESTRING (12.5191663 55.664032, 12.519428 55...
Name: geometry, Length: 19559, dtype: object

In [6]:
s2 = pd.merge(edges, edges_simplified, how='left', on=['osmid'])

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [111]:
h= csv_to_ig_custom(PATH["data"] + "copenhagen" + "/", "copenhagen", "biketrack", "pop_den_attr")

In [112]:
calculate_efficiency_global(h)

113.05329180359571

In [35]:
hh = csv_to_ig_custom(PATH["data"] + "copenhagen" + "/", "copenhagen", "biketrack", "bcount_attr")

In [114]:
calculate_efficiency_global(hh)

100.22215376444066

In [36]:
hhh = csv_to_ig_custom(PATH["data"] + "copenhagen" + "/", "copenhagen", "carall_simplified","length_attr")

TypeError: type str doesn't define __round__ method

In [116]:
calculate_efficiency_global(hhh)

100.58950061928145

In [34]:
hhhh = csv_to_ig_custom(PATH["data"] + "copenhagen" + "/", "copenhagen", "biketrack_simplified", "length")

In [ ]:
hhhh = csv_to_ig_custom(PATH["data"] + "copenhagen" + "/", "copenhagen", "biketrack", "length_attr")

In [118]:
calculate_efficiency_global(hhhh)

70.06177909807477

## Analyze existing infrastructure

In [119]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing existing infrastructure.")
    
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    for attr in attrlist:
        filename = placeid + "_"+attr +"_existing.csv"
        if rerun_existing or not os.path.isfile(PATH["results"] + placeid + "/" + filename):
            empty_metrics = {
                             "length":0,
                             #"length_lcc":0,
                             "coverage": 0,
                             "directness": 0,
                             #"directness_lcc": 0,
                             #"poi_coverage": 0,
                             #"components": 0,
                             #"efficiency_global": 0,
                             #"efficiency_local": 0,
                             #"efficiency_global_routed": 0,
                             #"efficiency_local_routed": 0,
                             #"directness_lcc_linkwise": 0,
                             #"directness_all_linkwise": 0
                            }
            output_place = {}
            for networktype in networktypes:
                output_place[networktype] = copy.deepcopy(empty_metrics)

            # Analyze all networks     
            Gs = {}
            for networktype in networktypes:
                if networktype != "biketrack_onstreet" and networktype != "bikeable_offstreet":
                    Gs[networktype] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype,attr)
                    Gs[networktype + "_simplified"] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype + "_simplified",attr)
                    #Gs[networktype] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype)
                    #Gs[networktype + "_simplified"] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
                elif networktype == "biketrack_onstreet":
                    Gs[networktype] = intersect_igraphs(Gs["biketrack"], Gs["carall"])
                    Gs[networktype + "_simplified"] = intersect_igraphs(Gs["biketrack_simplified"], Gs["carall_simplified"])
                elif networktype == "bikeable_offstreet":
                    G_temp = copy.deepcopy(Gs["bikeable"])
                    delete_overlaps(G_temp, Gs["carall"])
                    Gs[networktype] = G_temp
                    G_temp = copy.deepcopy(Gs["bikeable_simplified"])
                    delete_overlaps(G_temp, Gs["carall_simplified"])
                    Gs[networktype + "_simplified"] = G_temp

            with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
                nnids = [int(line.rstrip()) for line in f]


            covs = {}
            for networktype in tqdm(networktypes, desc = "Networks", leave = False):
                if debug: print(placeid + ": Analyzing results: " + networktype)
                metrics, cov = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)
                #metrics, cov = calculate_metrics_custom(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)

                for key, val in metrics.items():
                    output_place[networktype][key] = val
                covs[networktype] = cov
            # Save the covers
            write_result(covs, "pickle", placeid, "", "",attr+"_"+"existing_covers.pickle")

            # Write to CSV
            write_result(output_place, "dictnested", placeid, "", "", attr+"_"+"existing.csv", empty_metrics)

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

copenhagen: Analyzing existing infrastructure.


KeyError: 'length_attr'

## Analyze POI based results

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing results")
    for attr in attrlist:
        print("attr: " + attr)
    # Load networks
        G_carall = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, 'carall',attr)
        #G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
        Gexisting = {}
        for networktype in ["biketrack", "bikeable"]:
            Gexisting[networktype] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype, attr)



        # Load POIs
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

        # Load results
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_"+ attr
        resultfile = open(PATH["results"] + placeid + "/" + filename  +".pickle",'rb')
        res = pickle.load(resultfile)
        resultfile.close()
        if debug: pp.pprint(res)

        # Calculate
        # output contains lists for all the prune_quantile values of the corresponding results
        #output, covs = calculate_metrics_additively_custom(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
        #output_MST, cov_MST = calculate_metrics_custom(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)

        output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
        output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)

        # Save the covers
        write_result(covs, "pickle", placeid, poi_source, prune_measure,  "_"+ attr+"_covers.pickle")
    #     write_result(covs_carminusbike, "pickle", placeid, poi_source, prune_measure, "_covers_carminusbike.pickle")
        write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_"+ attr+"_cover_mst.pickle")

        # Write to CSV
        write_result(output, "dict", placeid, poi_source, prune_measure, "_"+ attr+".csv")
    #     write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
    #     write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")
        write_result(output_MST, "dict", placeid, poi_source, "", "_"+ attr+"mst.csv")

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

copenhagen: Analyzing results
attr: length


In [ ]:
# vi skal lave en test hvor vi får den til at lave en folder inde i plotsnewtork for length attr
#vi skal lave et outer loop for hver attr inden Gs = {}, og gemme en folder for hver attr.
# vi skal joine de 3 attributes fra carall netværket med de gamle networks?
# opdater notebook 5 og se om det virker med at lave plots på attribute,

In [ ]:
Audio(sound_file, autoplay=True) 